<a href="https://colab.research.google.com/github/kmu-kwonsoonbeom/25_S_SWP1/blob/main/Report1_20222057_%EC%A0%95%EB%B3%B4%EB%B3%B4%EC%95%88%EC%95%94%ED%98%B8%EC%88%98%ED%95%99%EA%B3%BC_%EA%B6%8C%EC%88%9C%EB%B2%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def read_matrix():
    n = int(input("정방행렬의 차수 n을 입력하세요: ").strip())
    if n <= 0:
        raise ValueError("n은 양의 정수여야 합니다.")

    M = []
    print(f"{n}x{n} 행렬을 입력하세요 (각 행은 공백으로 구분):")

    for i in range(1, n + 1):
        row = input(f"{i}번째 행: ").split()
        if len(row) != n:
            raise ValueError(f"{i}번째 행에 {n}개가 아닌 {len(row)}개가 입력되었습니다.")
        M.append([float(x) for x in row])
    return M

def print_matrix(M, title=None, precision=6, box=False):
    if title:
        print(f"=== {title} ===")
    if box:
      fmt = f"%{precision+4}.{precision}f"
      size = len(M)
      print("┌" + " " * 8 * size + "┐")
      for i in range(size):
        print("│", end=" ")
        for j in range(size):
          print(fmt % M[i][j], end=" ")
        print("│")
      print("└" + " " * 8 * size + "┘")

def identity_matrix(n):
    return [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]

def deep_copy(M):
    return [row[:] for row in M]

def matrix_equal(A, B, eps=1e-8):
    if A is None or B is None:
        return False
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        return False
    n, m = len(A), len(A[0])
    for i in range(n):
        for j in range(m):
            if abs(A[i][j] - B[i][j]) > eps:
                return False
    return True


In [ ]:
def getMatrixMinor(m, i, j):
  return [row[:j] + row[j+1:] for row in (m[:i] + m[i+1:])]

def transposeMatrix(m):
  return [[m[j][i] for j in range(len(m))] for i in range(len(m[0]))]

def getMatrixDeterminant(m):
  n = len(m)
  if n == 1:
    return m[0][0]
  if n == 2:
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

  det = 0.0
  for c in range(n):
    det += ((-1)**c) * m[0][c] * getMatrixDeterminant(getMatrixMinor(m, 0, c))
  return det


In [ ]:
def inverse_by_adjugate(m, eps=1e-12):
    n = len(m)
    det = getMatrixDeterminant(m)
    if abs(det) < eps:
        raise ZeroDivisionError("행렬식이 0이므로 (행렬식 방법) 역행렬이 존재하지 않습니다.")

    if n == 1:
        return [[1.0 / m[0][0]]]
    if n == 2:
        return [
            [ m[1][1] / det, -m[0][1] / det],
            [-m[1][0] / det,  m[0][0] / det]
        ]

    cof = []
    for i in range(n):
        row = []
        for j in range(n):
            minor = getMatrixMinor(m, i, j)
            row.append(((-1)**(i+j)) * getMatrixDeterminant(minor))
        cof.append(row)
    adj = transposeMatrix(cof)
    return [[adj[i][j] / det for j in range(n)] for i in range(n)]


In [ ]:
def inverse_by_gauss_jordan(m, eps=1e-12):
    if len(m) != len(m[0]):
        raise ValueError("정방행렬만 역행렬을 구할 수 있습니다.")
    n = len(m)
    A = deep_copy(m)
    I = identity_matrix(n)

    for col in range(n):
        pivot_row = max(range(col, n), key=lambda r: abs(A[r][col]))
        if abs(A[pivot_row][col]) < eps:
            raise ZeroDivisionError("피벗이 0이 되어 (가우스-조던) 역행렬이 존재하지 않습니다.")

        if pivot_row != col:
            A[col], A[pivot_row] = A[pivot_row], A[col]
            I[col], I[pivot_row] = I[pivot_row], I[col]

        pivot = A[col][col]
        for j in range(n):
            A[col][j] /= pivot
            I[col][j] /= pivot

        for r in range(n):
            if r == col:
                continue
            factor = A[r][col]
            if abs(factor) < eps:
                continue
            for j in range(n):
                A[r][j] -= factor * A[col][j]
                I[r][j] -= factor * I[col][j]

    return I


In [ ]:
def main():
    try:
        A = read_matrix()

        inv_adj = None
        try:
            inv_adj = inverse_by_adjugate(A)
            print_matrix(inv_adj, "역행렬 (행렬식/수반행렬)", precision=3, box=True)
        except Exception as e:
            print(f"[행렬식 방법] 오류: {e}")

        inv_gj = None
        try:
            inv_gj = inverse_by_gauss_jordan(A)
            print_matrix(inv_gj, "역행렬 (가우스-조던)", precision=3, box=True)
        except Exception as e:
            print(f"[가우스-조던 방법] 오류: {e}")

        if inv_adj is not None and inv_gj is not None:
            same = matrix_equal(inv_adj, inv_gj, eps=1e-8)
            msg = "두 방법의 결과가 동일합니다." if same else "두 방법의 결과가 서로 다릅니다."
            print("비교 결과:", msg)

    except Exception as e:
        print(f"[입력/실행 오류] {e}")

if __name__ == "__main__":
    main()

정방행렬의 차수 n을 입력하세요: 2
2x2 행렬을 입력하세요 (각 행은 공백으로 구분):
1번째 행: 2 3
2번째 행: 1 4
=== 역행렬 (행렬식/수반행렬) ===
┌                ┐
│   0.800  -0.600 │
│  -0.200   0.400 │
└                ┘
=== 역행렬 (가우스-조던) ===
┌                ┐
│   0.800  -0.600 │
│  -0.200   0.400 │
└                ┘
비교 결과: 두 방법의 결과가 동일합니다.
